In [1]:
# from ColorExtractor import post
from ColorExtractor import tools
import re
import typer
res = [int(s) for s in re.findall(
    r'\d+', "8756_8768_12124_12137_z15_t110400")]
# print(res)
zoom = res[4]
# print(res[0], res[2])
# print(res[1], res[3])
NW_lat, NW_lng = tools.num2deg(res[0], res[2], zoom)
SE_lat, SE_lng = tools.num2deg(res[1], res[3], zoom)
print(NW_lat, NW_lng)
print(SE_lat, SE_lng)

# Estabilish the size of the image, Northwest as zero point
X = SE_lat - NW_lat
Y = SE_lng - NW_lng

print(X)
print(Y)

42.32606244456203 -83.8037109375
42.22038178372061 -83.671875
-0.10568066084141492
0.1318359375


In [2]:
import pandas as pd
filename = 'VED_171101_week'
path = "/Users/alexwell/Desktop/DAT295-Road-pattern-matching/"
_input = pd.read_csv(path + 'ved_data_enrichment/data/ved-final/'+filename+'.csv',
                             dtype={"Matchted Latitude[deg]": float, 'Matched Longitude[deg]': float,
                                    'Vehicle Speed[km/h]': float, 'Speed Limit[km/h]': 'string'},
                             nrows=1000)

In [3]:
_input.head()

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],MAF[g/sec],Engine RPM[RPM],Absolute Load[%],...,Energy_Consumption,Matchted Latitude[deg],Matched Longitude[deg],Match Type,Class of Speed Limit,Speed Limit[km/h],Speed Limit with Direction[km/h],Intersection,Focus Points,Bus Stops
0,1.586651,8.0,706.0,0.0,42.277558,-83.69875,40.0,22.129999,2285.0,49.019608,...,NaN,42.277558,-83.698767,0,0.0,56,56.0,NaN,NaN,NaN
1,1.586651,8.0,706.0,200.0,42.277558,-83.69875,40.0,22.129999,2285.0,67.450981,...,NaN,42.277558,-83.698767,1,0.0,56,56.0,NaN,NaN,NaN
2,1.586651,8.0,706.0,1100.0,42.277558,-83.69875,45.0,22.129999,2285.0,67.450981,...,NaN,42.277558,-83.698767,1,0.0,56,56.0,NaN,NaN,NaN
3,1.586651,8.0,706.0,2100.0,42.277558,-83.69875,47.0,6.150000,2744.0,67.450981,...,NaN,42.277558,-83.698767,1,0.0,56,56.0,NaN,NaN,NaN
4,1.586651,8.0,706.0,4200.0,42.277558,-83.69875,48.0,21.440001,1982.0,67.450981,...,NaN,42.277558,-83.698767,1,0.0,56,56.0,NaN,NaN,NaN


In [11]:
path = "/Users/alexwell/Desktop/DAT295-Road-pattern-matching/"
manual_sample = pd.read_csv('/Users/alexwell/Desktop/DAT295-Road-pattern-matching/ColorExtractor/manual_selected_samples.csv')
samples_groupby_filename = manual_sample.groupby('Filename')
    
new_accuract = 0
old_accuract = 0
total = 0
for filename, group in samples_groupby_filename:
    # Read the data
    _input = pd.read_csv(path+'ved_data_enrichment/data/ved-final/'+filename+'.csv',
                                dtype={"Latitude[deg]": float, 'Longitude[deg]': float,
                                    'Vehicle Speed[km/h]': float, 'Speed Limit[km/h]': 'string'},
                                nrows=10000)
    _input['Index'] = _input['DayNum'].apply(tools.timeCalc)
    # sample index in this file
    sample_index = group['index'].tolist()
    print(sample_index)
    for index, row in _input.iterrows():
        up_limit = 30
        longitude_seq = []
        latitude_seq = []
        
        if index in sample_index:
            for i in range(up_limit):
                # Change every 6 items
                new_index = index
                lng_val = (_input['Longitude[deg]'].at[index - i] - NW_lng)/Y
                lat_val = (_input['Latitude[deg]'].at[index - i] - NW_lat)/X
                # adptiv track length by delta
                longitude_seq.append(lng_val)
                latitude_seq.append(lat_val)
                idx = _input['Index'].at[index]
                rest_point_delta, max_delta = tools.point_delta(longitude_seq, latitude_seq, idx)
                print(rest_point_delta, max_delta)
                if max_delta > 20:
                    break
            color_code = tools.image_process_position_seq(longitude_seq, latitude_seq, idx, rest_point_delta, max_delta)
        break   


[496, 516, 1241, 1321, 1621, 1927, 2147, 2319, 2339, 2439, 2639, 3019, 3101, 3121, 3401, 3796, 3856, 3896, 4316, 4576, 5840, 6060, 6120, 6840, 7020, 7230, 8187, 8647]
[919, 1324]


In [5]:
# Show trajectory
long_seq = []
lat_seq = []

matched = True

print(grouped)
target_group = 2
groupID = 0
for name, group in grouped:
    if groupID == target_group:
        for index, row in group.iterrows():
            # print(index)
            if matched:
                lng_val = (_input['Matched Longitude[deg]'].at[index] - NW_lng)/Y
                lat_val = (_input['Matchted Latitude[deg]'].at[index] - NW_lat)/X
            else:
                lng_val = (_input['Longitude[deg]'].at[index] - NW_lng)/Y
                lat_val = (_input['Latitude[deg]'].at[index] - NW_lat)/X
            long_seq.append(lng_val)
            lat_seq.append(lat_val)
    groupID += 1
        

tools.show_trajectory(long_seq, lat_seq, index)

NameError: name 'grouped' is not defined